In [2]:
import os
import re
import pandas as pd
import numpy as np

In [6]:
data=pd.read_csv("E:\git\dsmp-2024-group22\data_preprocess\DATA\processed_tapes.csv",index_col="date")
data.drop(["Unnamed: 0"],axis=1)

,max_price,min_price,open_price,close_price
date,,,,
2025-01-02,303.0,243.0,267.0,288.0
2025-01-03,317.0,270.0,281.0,291.0
2025-01-04,NaN,NaN,NaN,NaN
2025-01-05,NaN,NaN,NaN,NaN
2025-01-06,346.0,276.0,278.0,327.0
...,...,...,...,...
2025-06-27,113.0,93.0,106.0,111.0
2025-06-28,NaN,NaN,NaN,NaN
2025-06-29,NaN,NaN,NaN,NaN


In [4]:
dir_path = r"E:\BRISTOL COURSE\TB2\mini project bristol\JPMorgan_Set01\Tapes"
file_ls = os.listdir(dir_path)  #get every name of the file
file_ls[:5]

['UoB_Set01_2025-01-02tapes.csv',
 'UoB_Set01_2025-01-03tapes.csv',
 'UoB_Set01_2025-01-06tapes.csv',
 'UoB_Set01_2025-01-07tapes.csv',
 'UoB_Set01_2025-01-08tapes.csv']

In [5]:
len(file_ls)

125

In [4]:
#found the max and min deal price
def detailprice(name):
    date = []
    segments = []
    tapes = pd.read_csv(r"E:/BRISTOL COURSE/TB2/mini project bristol/JPMorgan_Set01/Tapes/" + name + "", header=None,
                        names=["Times", "price", "volume"])
    tapes['price'] = pd.to_numeric(tapes['price'], errors='coerce')
    segment_size = len(tapes) // 100
    data_pattern = r'\d{4}-\d{2}-\d{2}'
    match = re.search(data_pattern, name)
    date.append(match.group())
    for i in range(50):
        start_idx = i * segment_size
        # 对于最后一个段，取所有剩余的数据点
        end_idx = start_idx + segment_size if i < 49 else len(tapes)

        segment = tapes.iloc[start_idx:end_idx]
        max_price = segment['price'].max()
        min_price = segment['price'].min()
        open_price = segment['price'].iloc[0]
        close_price = segment['price'].iloc[-1]
        cumsum = segment["volume"].sum()
        segments.append({
            'period': date[0] + f"-{i + 1}",
            'max_price': max_price,
            'min_price': min_price,
            'open_price': open_price,
            'close_price': close_price,
            'Volume': cumsum
        })
        segments_df = pd.DataFrame(segments)
    return segments_df

In [9]:
#found the max and min deal price
def detailVolume(name):
    segments = []
    tapes = pd.read_csv(r"E:/BRISTOL COURSE/TB2/mini project bristol/JPMorgan_Set01/Tapes/" + name + "", header=None,
                        names=["Times", "price", "volume"])
    cumsum = tapes["volume"].sum()
    segments.append({'Volume': cumsum})
    v_df = pd.DataFrame(segments)
    return v_df

result_df=pd.DataFrame(columns=['Volume'])
df_list = []
for i in file_ls:
    segments_df = detailVolume(i)  # 调用detailprice函数处理每个文件
    df_list.append(segments_df)  # 将结果添加到列表中
# 在循环结束后，一次性拼接所有DataFrame
volume_df = pd.concat(df_list, axis=0, ignore_index=True)  # 使用ignore_index=True重置索引
# 现在 result_df 包含了所有文件处理后的数据，无需再创建新的DataFrame
volume_df.to_csv("E:\git\dsmp-2024-group22\data_preprocess\DATA/tapes_volume.csv")

In [5]:
# 初始空DataFrame，用于定义列名
result_df = pd.DataFrame(columns=['period', 'max_price', 'min_price', 'open_price', 'close_price', 'Volume'])

# 用于存储每个文件处理结果的DataFrame列表
df_list = []
for i in file_ls:
    segments_df = detailprice(i)  # 调用detailprice函数处理每个文件
    df_list.append(segments_df)  # 将结果添加到列表中
# 在循环结束后，一次性拼接所有DataFrame
result_df = pd.concat(df_list, axis=0, ignore_index=True)  # 使用ignore_index=True重置索引
# 现在 result_df 包含了所有文件处理后的数据，无需再创建新的DataFrame
result_df

,period,max_price,min_price,open_price,close_price,Volume
0,2025-01-02-1,270,250,267,266,458
1,2025-01-02-2,269,249,263,261,497
2,2025-01-02-3,269,249,264,261,446
3,2025-01-02-4,269,248,268,266,462
4,2025-01-02-5,270,250,269,255,466
...,...,...,...,...,...,...
12495,2025-07-01-96,113,93,105,107,552
12496,2025-07-01-97,113,93,107,102,532
12497,2025-07-01-98,112,93,93,112,537
12498,2025-07-01-99,112,93,112,107,537


In [6]:
result_df.to_csv("E:\git\dsmp-2024-group22\data_preprocess\DATA\sperated_tapes_100.csv")

In [7]:
data_pattern = r'\d{4}-\d{2}-\d{2}'  # found the date in the file
date = []
for i in file_ls:
    match = re.search(data_pattern, i)
    date.append(match.group())
date = pd.to_datetime(date)
date

DatetimeIndex(['2025-01-02', '2025-01-03', '2025-01-06', '2025-01-07',
               '2025-01-08', '2025-01-09', '2025-01-10', '2025-01-13',
               '2025-01-14', '2025-01-15',
               ...
               '2025-06-18', '2025-06-19', '2025-06-20', '2025-06-23',
               '2025-06-24', '2025-06-25', '2025-06-26', '2025-06-27',
               '2025-06-30', '2025-07-01'],
              dtype='datetime64[ns]', length=125, freq=None)

In [8]:
file_ls_series = pd.Series(file_ls)
results_df = []


def apply_detailprice(filename):
    return detailprice(filename)  # Make sure that this function returns a tuple or list of four values


results_df = file_ls_series.apply(apply_detailprice).apply(pd.Series)
results_df.columns = ['max_price', 'min_price', 'open_price', 'close_price']
date = pd.Series(date)
results_df.insert(0, "date", date)
results_df.head(5)



ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
start_date, end_date = date.min(), date.max()  # create a new dataframe which have all date in this range
date_range = pd.date_range(start=start_date, end=end_date)
date_range = pd.Series(date_range)
df_full = pd.DataFrame(date_range, columns=['date'])
df_full.head(5)

In [ ]:
df_merged = pd.merge(df_full, results_df, on="date", how="left")  # left join 2 df
df_merged.head(5)

In [12]:
df_merged.to_csv("E:/mini project bristol/processed data(LOBS,tapes)/processed_tapes.csv")

NameError: name 'df_merged' is not defined